In [ ]:
! pip install tensorflow

In [16]:
! pip install scikit-learn

  Using cached scikit_learn-1.4.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.13.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.4.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.4.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.4.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.2 MB)
Using cached joblib-1.4.0-py3-none-any.whl (301 kB)
Using cached scipy-1.13.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.2 MB)
Using cached threadpoolctl-3.4.0-py3-none-any.whl (17 kB)


In [15]:
import numpy as np
import pandas as pd
import warnings

In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential

2024-04-13 09:54:17.382548: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 09:54:17.386158: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 09:54:17.422563: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 09:54:18.425591: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
import os
directory = "nsl-kdd/"
document_counts = {}

for filename in os.listdir(directory):
    if filename.endswith(".txt")  :
        document_counts[filename] = len(open(os.path.join(directory, filename), 'r').readlines())
document_counts 

{'KDDTest+.txt': 22544,
 'KDDTest-21.txt': 11850,
 'KDDTrain+.txt': 125973,
 'KDDTrain+_20Percent.txt': 25192}

In [8]:
file_train_name = 'KDDTrain+.txt'
file_test_name = 'KDDTest+.txt'
column_names = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 
                'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 
                'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 
                'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 
                'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 
                'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'sub_label', 'level']
column_drops = ['level']

In [9]:
train_df = pd.read_csv(os.path.join(directory, file_train_name), header=None, names=column_names
                      , usecols=lambda x: x not in column_drops)
test_df = pd.read_csv(os.path.join(directory, file_test_name),header=None, names=column_names
                      , usecols=lambda x: x not in column_drops)
df_full = pd.concat([train_df, test_df], ignore_index=True)
df_full = pd.concat([train_df, test_df])
print(f"Shape train: {train_df.shape}")
print(f"Shape test: {test_df.shape}")
df_full

Shape train: (125973, 42)
Shape test: (22544, 42)


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,sub_label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal
22540,0,tcp,http,SF,317,938,0,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,back
22542,0,udp,domain_u,SF,42,42,0,0,0,0,...,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [10]:
attack_categories = {
    'normal': ['normal'],
    'DoS': ['neptune', 'satan', 'smurf', 'apache2', 'back', 'mailbomb', 'teardrop', 'pod', 'land', 'worm', 'udpstorm'],
    'Probe': ['mscan', 'ipsweep', 'portsweep', 'nmap', 'snmpguess', 'saint'],
    'U2R': ['guess_passwd', 'processtable', 'buffer_overflow', 'multihop', 'rootkit', 'named', 'ps', 'sendmail', 'xterm', 'xlock', 'phf', 'loadmodule', 'perl', 'sqlattack', 'spy'],
    'R2L': ['warezmaster', 'snmpgetattack', 'httptunnel', 'warezclient', 'imap', 'ftp_write', 'xsnoop']
}
dict_attack_outcome = {}
for clas, subclass in attack_categories.items():
    for subclas in subclass:
        dict_attack_outcome[subclas] = clas
dict_attack_outcome

{'normal': 'normal',
 'neptune': 'DoS',
 'satan': 'DoS',
 'smurf': 'DoS',
 'apache2': 'DoS',
 'back': 'DoS',
 'mailbomb': 'DoS',
 'teardrop': 'DoS',
 'pod': 'DoS',
 'land': 'DoS',
 'worm': 'DoS',
 'udpstorm': 'DoS',
 'mscan': 'Probe',
 'ipsweep': 'Probe',
 'portsweep': 'Probe',
 'nmap': 'Probe',
 'snmpguess': 'Probe',
 'saint': 'Probe',
 'guess_passwd': 'U2R',
 'processtable': 'U2R',
 'buffer_overflow': 'U2R',
 'multihop': 'U2R',
 'rootkit': 'U2R',
 'named': 'U2R',
 'ps': 'U2R',
 'sendmail': 'U2R',
 'xterm': 'U2R',
 'xlock': 'U2R',
 'phf': 'U2R',
 'loadmodule': 'U2R',
 'perl': 'U2R',
 'sqlattack': 'U2R',
 'spy': 'U2R',
 'warezmaster': 'R2L',
 'snmpgetattack': 'R2L',
 'httptunnel': 'R2L',
 'warezclient': 'R2L',
 'imap': 'R2L',
 'ftp_write': 'R2L',
 'xsnoop': 'R2L'}

In [11]:
df_full['label'] = df_full.sub_label.apply(lambda x: dict_attack_outcome[x])
train_df['label'] = train_df.sub_label.apply(lambda x: dict_attack_outcome[x])

In [12]:
X, y = df_full.drop(['label', 'sub_label'], axis=1), df_full[['label']]

In [13]:
columns_object_X = X.select_dtypes(include=['object']).columns

In [17]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output =False).set_output(transform="pandas")
X_oh = ohe.fit_transform(X[columns_object_X])
X.drop(columns_object_X, axis=1, inplace=True)
X = pd.concat([X, X_oh], axis=1)

In [18]:
from sklearn.preprocessing import MinMaxScaler

def normalize_with_sklearn(df, cols):
    result = df.copy() 
    scaler = MinMaxScaler()
    result[cols] = scaler.fit_transform(df[cols])
    return result
def normalize_with_log(df, cols):
    result = df.copy() 
    for col in cols:
        result[col] = np.log1p(result[col])
    return result

In [19]:
columns_with_values_other_than_zero_or_one = X.columns[~((X == 0) | (X == 1)).all()]

In [20]:
X = normalize_with_log(X, columns_with_values_other_than_zero_or_one)

In [21]:
lohe = OneHotEncoder(sparse_output =False).set_output(transform="pandas")
lohe.fit_transform(y)
lohe.categories_

[array(['DoS', 'Probe', 'R2L', 'U2R', 'normal'], dtype=object)]

In [22]:
from sklearn.utils.class_weight import compute_class_weight


y_train_series = df_full['label'].squeeze()
class_labels = np.unique(y_train_series)
class_weights = compute_class_weight(class_weight='balanced', 
                                     classes=class_labels, y=y_train_series)
class_weight_dict = dict(zip(class_labels, class_weights))
class_weight_dict

{'DoS': 0.5204731032065884,
 'Probe': 2.9585059760956174,
 'R2L': 13.550821167883212,
 'U2R': 13.745210550670986,
 'normal': 0.3854880992550679}

In [23]:
class_weight_dict_number = {0: 0.5204731032065884,
 1: 2.9585059760956174,
 2: 13.550821167883212,
 3: 13.745210550670986,
 4: 0.3854880992550679}

In [24]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
kfold.get_n_splits(X,y)

10

In [25]:
for train_index, test_index in kfold.split(X,y):
    train_X_new, test_X_new = X.iloc[train_index], X.iloc[test_index]
    train_y_new, test_y_new = y.iloc[train_index], y.iloc[test_index]
    print(train_X_new.shape, test_X_new.shape)

(133665, 122) (14852, 122)
(133665, 122) (14852, 122)
(133665, 122) (14852, 122)
(133665, 122) (14852, 122)
(133665, 122) (14852, 122)
(133665, 122) (14852, 122)
(133665, 122) (14852, 122)
(133666, 122) (14851, 122)
(133666, 122) (14851, 122)
(133666, 122) (14851, 122)


In [26]:
model = Sequential([
    layers.Input(shape=(X.shape[1],1)),

    layers.Conv1D(32, kernel_size=(5), strides=1, activation='relu'),
    layers.MaxPooling1D(pool_size=(5),padding='same'),

    layers.Conv1D(64, kernel_size=(5), strides=1, activation='relu'),
    layers.MaxPooling1D(pool_size=(5),padding='same'),

    layers.Flatten(),

    layers.Dense(500),

    layers.Dense(500),
    
    layers.Dropout(0.5),

    layers.Dense(5, activation='softmax')
])

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy',
              metrics=['accuracy']
             )

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 118, 32)        │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 24, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 20, 64)         │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 500)            │       128,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 500)            │       250,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │         2,505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 392,001 (1.50 MB)

 Trainable params: 392,001 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
list_model_cnn = []

In [28]:
from sklearn.metrics import f1_score, recall_score, accuracy_score

In [29]:
his_acc, his_f1, his_rc = [], [], []
for i, (train_index, test_index) in enumerate(kfold.split(X,y)):
    print(f"Fold {i+1} --------------------------------")
    train_X_new, test_X_new = X.iloc[train_index], X.iloc[test_index]
    train_y_new, test_y_new = y.iloc[train_index], y.iloc[test_index]

    train_y_new = lohe.transform(train_y_new)
    test_y_new = lohe.transform(test_y_new)
    
    his = model.fit(
        train_X_new, train_y_new,
        validation_data=(test_X_new, test_y_new), 
        batch_size=32, 
        epochs=10,
        class_weight=class_weight_dict_number
    )
    
    pred = model.predict(test_X_new)
    pred = np.argmax(pred,axis=1)
    y_eval = np.argmax(test_y_new,axis=1)
    
    acc = []
    for class_label in sorted(set(y_eval)):
        indices = y_eval == class_label
        acc.append(accuracy_score(y_eval[indices], pred[indices]))

    f1 = f1_score(y_eval, pred, average=None)
    recall = recall_score(y_eval, pred, average=None)
    his_acc.append(acc)
    his_f1.append(f1)
    his_rc.append(recall)
    
    print("Accuracy score:", accuracy_score(y_eval, pred))
    print("F1 Score:", f1_score(y_eval, pred, average='weighted'))
    print("Recall:", recall_score(y_eval, pred, average='weighted'))
    list_model_cnn.append(model)

Fold 1 --------------------------------
Epoch 1/10
4178/4178 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.8733 - loss: 0.3694 - val_accuracy: 0.9179 - val_loss: 0.2522
Epoch 2/10
4178/4178 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.9366 - loss: 0.1724 - val_accuracy: 0.9259 - val_loss: 0.3093
Epoch 3/10
4178/4178 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.9452 - loss: 0.1469 - val_accuracy: 0.9280 - val_loss: 0.2300
Epoch 4/10
4178/4178 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9517 - loss: 0.1280 - val_accuracy: 0.9446 - val_loss: 0.1916
Epoch 5/10
4178/4178 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.9557 - loss: 0.1165 - val_accuracy: 0.9667 - val_loss: 0.0865
Epoch 6/10
4178/4178 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9587 - loss: 0.1112 - val_accuracy: 0.9258 - val_loss: 0.2192
Epoch 7/10
4178/4178 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9598 - loss: 0.1155 - val_accuracy: 0.9803 - val_loss: 0.0627
Epoch 8/10
4178/4178 ━━━━━━━━━━━━━━━━━━━━ 1

In [30]:
best_model_index = 5
best_model = list_model_cnn[best_model_index]

In [31]:
model.save('cnnModel.h5')